⊕ [Migrating CSV, JSON and XML Data with Client Python | GRAKN.AI](https://dev.grakn.ai/docs/examples/phone-calls-migration-python)

⊕ [Cannot create Grakn Table · Issue #4 · RasaHQ/tutorial-knowledge-base](https://github.com/RasaHQ/tutorial-knowledge-base/issues/4)


In [25]:
from grakn.client import GraknClient
with GraknClient(uri="sagas:48555") as client:
    client.keyspaces().delete("phone_calls")

# $ grakn console --keyspace phone_calls --file files/phone-calls/schema.gql

In [28]:
from grakn.client import GraknClient
import csv

def build_phone_call_graph(inputs):
    with GraknClient(uri="sagas:48555") as client:
        with client.session(keyspace = "phone_calls") as session:
            for input in inputs:
                print("Loading from [" + input["data_path"] + "] into Grakn ...")
                load_data_into_grakn(input, session)

def load_data_into_grakn(input, session):
    items = parse_data_to_dictionaries(input)

    for item in items:
        with session.transaction().write() as transaction:
            graql_insert_query = input["template"](item)
            print("Executing Graql Query: " + graql_insert_query)
            transaction.query(graql_insert_query)
            transaction.commit()

    print("\nInserted " + str(len(items)) + " items from [ " + input["data_path"] + "] into Grakn.\n")

def company_template(company):
    return 'insert $company isa company, has name "' + company["name"] + '";'

def person_template(person):
    # insert person
    graql_insert_query = 'insert $person isa person, has phone-number "' + person["phone_number"] + '"'
    if person["first_name"] == "":
        # person is not a customer
        graql_insert_query += ", has is-customer false"
    else:
        # person is a customer
        graql_insert_query += ", has is-customer true"
        graql_insert_query += ', has first-name "' + person["first_name"] + '"'
        graql_insert_query += ', has last-name "' + person["last_name"] + '"'
        graql_insert_query += ', has city "' + person["city"] + '"'
        graql_insert_query += ", has age " + str(person["age"])
    graql_insert_query += ";"
    return graql_insert_query

def contract_template(contract):
    # match company
    graql_insert_query = 'match $company isa company, has name "' + contract["company_name"] + '";'
    # match person
    graql_insert_query += ' $customer isa person, has phone-number "' + contract["person_id"] + '";'
    # insert contract
    graql_insert_query += " insert (provider: $company, customer: $customer) isa contract;"
    return graql_insert_query

def call_template(call):
    # match caller
    graql_insert_query = 'match $caller isa person, has phone-number "' + call["caller_id"] + '";'
    # match callee
    graql_insert_query += ' $callee isa person, has phone-number "' + call["callee_id"] + '";'
    # insert call
    graql_insert_query += (" insert $call(caller: $caller, callee: $callee) isa call; " +
                           "$call has started-at " + call["started_at"] + "; " +
                           "$call has duration " + str(call["duration"]) + ";")
    return graql_insert_query

def parse_data_to_dictionaries(input):
    items = []
    with open(input["data_path"] + ".csv") as data: # 1
        for row in csv.DictReader(data, skipinitialspace = True):
            item = { key: value for key, value in row.items() }
            items.append(item) # 2
    return items

In [29]:
inputs = [
    {
        "data_path": "files/phone-calls/data/companies",
        "template": company_template
    },
    {
        "data_path": "files/phone-calls/data/people",
        "template": person_template
    },
    {
        "data_path": "files/phone-calls/data/contracts",
        "template": contract_template
    },
    {
        "data_path": "files/phone-calls/data/calls",
        "template": call_template
    }
]

build_phone_call_graph(inputs=inputs)

Loading from [files/phone-calls/data/companies] into Grakn ...
Executing Graql Query: insert $company isa company, has name "Telecom";

Inserted 1 items from [ files/phone-calls/data/companies] into Grakn.

Loading from [files/phone-calls/data/people] into Grakn ...
Executing Graql Query: insert $person isa person, has phone-number "+7 171 898 0853", has is-customer true, has first-name "Melli", has last-name "Winchcum", has city "London", has age 55;
Executing Graql Query: insert $person isa person, has phone-number "+370 351 224 5176", has is-customer true, has first-name "Celinda", has last-name "Bonick", has city "London", has age 52;
Executing Graql Query: insert $person isa person, has phone-number "+81 308 988 7153", has is-customer true, has first-name "Chryste", has last-name "Lilywhite", has city "London", has age 66;
Executing Graql Query: insert $person isa person, has phone-number "+54 398 559 0423", has is-customer true, has first-name "D'arcy", has last-name "Byfford", h

Executing Graql Query: match $caller isa person, has phone-number "+62 107 530 7500"; $callee isa person, has phone-number "+263 498 495 0617"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-25T22:58:02; $call has duration 5665;
Executing Graql Query: match $caller isa person, has phone-number "+54 398 559 0423"; $callee isa person, has phone-number "+86 892 682 0628"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-23T08:55:18; $call has duration 822;
Executing Graql Query: match $caller isa person, has phone-number "+7 690 597 4443"; $callee isa person, has phone-number "+54 398 559 0423"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-25T09:10:25; $call has duration 8494;
Executing Graql Query: match $caller isa person, has phone-number "+263 498 495 0617"; $callee isa person, has phone-number "+7 414 625 3019"; insert $call(caller: $caller, callee: $callee) isa call; 

Executing Graql Query: match $caller isa person, has phone-number "+7 690 597 4443"; $callee isa person, has phone-number "+351 515 605 7915"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-19T01:00:38; $call has duration 141;
Executing Graql Query: match $caller isa person, has phone-number "+81 308 988 7153"; $callee isa person, has phone-number "+63 808 497 1769"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-24T03:16:48; $call has duration 89;
Executing Graql Query: match $caller isa person, has phone-number "+261 860 539 4754"; $callee isa person, has phone-number "+86 892 682 0628"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-26T19:47:20; $call has duration 21;
Executing Graql Query: match $caller isa person, has phone-number "+7 690 597 4443"; $callee isa person, has phone-number "+48 697 447 6933"; insert $call(caller: $caller, callee: $callee) isa call; $cal

Executing Graql Query: match $caller isa person, has phone-number "+370 351 224 5176"; $callee isa person, has phone-number "+63 808 497 1769"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-16T20:26:23; $call has duration 2606;
Executing Graql Query: match $caller isa person, has phone-number "+263 498 495 0617"; $callee isa person, has phone-number "+86 921 547 9004"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-28T05:06:44; $call has duration 886;
Executing Graql Query: match $caller isa person, has phone-number "+81 308 988 7153"; $callee isa person, has phone-number "+54 398 559 0423"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-23T04:48:41; $call has duration 3458;
Executing Graql Query: match $caller isa person, has phone-number "+261 860 539 4754"; $callee isa person, has phone-number "+27 117 258 4149"; insert $call(caller: $caller, callee: $callee) isa cal

Executing Graql Query: match $caller isa person, has phone-number "+63 815 962 6097"; $callee isa person, has phone-number "+263 498 495 0617"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-19T23:16:49; $call has duration 2519;
Executing Graql Query: match $caller isa person, has phone-number "+81 746 154 2598"; $callee isa person, has phone-number "+1 254 875 4647"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-15T07:02:32; $call has duration 914;
Executing Graql Query: match $caller isa person, has phone-number "+62 107 530 7500"; $callee isa person, has phone-number "+81 308 988 7153"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-22T04:57:20; $call has duration 4455;
Executing Graql Query: match $caller isa person, has phone-number "+54 398 559 0423"; $callee isa person, has phone-number "+62 107 530 7500"; insert $call(caller: $caller, callee: $callee) isa call; 

Executing Graql Query: match $caller isa person, has phone-number "+261 860 539 4754"; $callee isa person, has phone-number "+48 697 447 6933"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-28T08:52:06; $call has duration 599;
Executing Graql Query: match $caller isa person, has phone-number "+7 690 597 4443"; $callee isa person, has phone-number "+86 921 547 9004"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-18T06:03:28; $call has duration 160;
Executing Graql Query: match $caller isa person, has phone-number "+370 351 224 5176"; $callee isa person, has phone-number "+7 552 196 4096"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-17T19:51:51; $call has duration 79;
Executing Graql Query: match $caller isa person, has phone-number "+62 107 530 7500"; $callee isa person, has phone-number "+7 171 898 0853"; insert $call(caller: $caller, callee: $callee) isa call; $cal

Executing Graql Query: match $caller isa person, has phone-number "+7 690 597 4443"; $callee isa person, has phone-number "+30 419 575 7546"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-24T01:11:29; $call has duration 1959;
Executing Graql Query: match $caller isa person, has phone-number "+63 815 962 6097"; $callee isa person, has phone-number "+48 697 447 6933"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-17T18:43:42; $call has duration 29;
Executing Graql Query: match $caller isa person, has phone-number "+62 107 530 7500"; $callee isa person, has phone-number "+86 922 760 0418"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-19T19:07:10; $call has duration 3196;
Executing Graql Query: match $caller isa person, has phone-number "+7 171 898 0853"; $callee isa person, has phone-number "+86 202 257 8619"; insert $call(caller: $caller, callee: $callee) isa call; $ca

Executing Graql Query: match $caller isa person, has phone-number "+63 815 962 6097"; $callee isa person, has phone-number "+1 254 875 4647"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-20T12:02:37; $call has duration 9;
Executing Graql Query: match $caller isa person, has phone-number "+81 308 988 7153"; $callee isa person, has phone-number "+48 894 777 5173"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-28T06:12:49; $call has duration 156;
Executing Graql Query: match $caller isa person, has phone-number "+370 351 224 5176"; $callee isa person, has phone-number "+351 515 605 7915"; insert $call(caller: $caller, callee: $callee) isa call; $call has started-at 2018-09-16T22:31:25; $call has duration 543;
Executing Graql Query: match $caller isa person, has phone-number "+7 690 597 4443"; $callee isa person, has phone-number "+48 697 447 6933"; insert $call(caller: $caller, callee: $callee) isa call; $cal

In [1]:
from grakn.client import GraknClient

with GraknClient(uri="sagas:48555") as client:
    with client.session(keyspace = "phone_calls") as session:
        with session.transaction().read() as transaction:
            query = [
                'match',
                '  $customer isa person, has phone-number $phone-number;',
                '  $company isa company, has name "Telecom";',
                '  (customer: $customer, provider: $company) isa contract;',
                '  $target isa person, has phone-number "+86 921 547 9004";',
                '  (caller: $customer, callee: $target) isa call, has started-at $started-at;',
                '  $min-date == 2018-09-14T17:18:49; $started-at > $min-date;',
                'get $phone-number;'
            ]

            print("\nQuery:\n", "\n".join(query))
            query = "".join(query)

            iterator = transaction.query(query)
            answers = iterator.collect_concepts()
            result = [ answer.value() for answer in answers ]

            print("\nResult:\n", result)


Query:
 match
  $customer isa person, has phone-number $phone-number;
  $company isa company, has name "Telecom";
  (customer: $customer, provider: $company) isa contract;
  $target isa person, has phone-number "+86 921 547 9004";
  (caller: $customer, callee: $target) isa call, has started-at $started-at;
  $min-date == 2018-09-14T17:18:49; $started-at > $min-date;
get $phone-number;

Result:
 ['+370 351 224 5176', '+54 398 559 0423', '+62 107 530 7500', '+7 690 597 4443', '+263 498 495 0617', '+81 308 988 7153', '+81 746 154 2598', '+63 815 962 6097']


In [34]:
with GraknClient(uri="sagas:48555") as client:
    with client.session(keyspace = "phone_calls") as session:
      with session.transaction().read() as transaction:
        query = [
          'match ',
          '  $suspect isa person, has city "London", has age > 50;',
          '  $company isa company, has name "Telecom";',
          '  (customer: $suspect, provider: $company) isa contract;',
          '  $pattern-callee isa person, has age < 20;',
          '  (caller: $suspect, callee: $pattern-callee) isa call, has started-at $pattern-call-date;',
          '  $target isa person, has phone-number $phone-number, has is-customer false;',
          '  (caller: $suspect, callee: $target) isa call, has started-at $target-call-date;',
          '  $target-call-date > $pattern-call-date;',
          'get $phone-number;'
        ]

        print("\nQuery:\n", "\n".join(query))
        query = "".join(query)

        iterator = transaction.query(query)
        answers = iterator.collect_concepts()
        result = [ answer.value() for answer in answers ]

        print("\nResult:\n", result)


Query:
 match 
  $suspect isa person, has city "London", has age > 50;
  $company isa company, has name "Telecom";
  (customer: $suspect, provider: $company) isa contract;
  $pattern-callee isa person, has age < 20;
  (caller: $suspect, callee: $pattern-callee) isa call, has started-at $pattern-call-date;
  $target isa person, has phone-number $phone-number, has is-customer false;
  (caller: $suspect, callee: $target) isa call, has started-at $target-call-date;
  $target-call-date > $pattern-call-date;
get $phone-number;

Result:
 ['+86 892 682 0628', '+86 202 257 8619', '+63 808 497 1769', '+86 922 760 0418', '+86 825 153 5518', '+27 117 258 4149', '+48 894 777 5173', '+351 515 605 7915', '+351 272 414 6570', '+30 419 575 7546', '+1 254 875 4647', '+33 614 339 0298']


In [35]:
client=GraknClient(uri="sagas:48555")
session=client.session(keyspace = "phone_calls")

In [40]:
with session.transaction().read() as transaction:
    query = [
      'match ',
      '  $suspect isa person, has city "London", has age > 50;',
      '  $company isa company, has name "Telecom";',
      '  (customer: $suspect, provider: $company) isa contract;',
      '  $pattern-callee isa person, has age < 20;',
      '  (caller: $suspect, callee: $pattern-callee) isa call, has started-at $pattern-call-date;',
      '  $target isa person, has phone-number $phone-number, has is-customer false;',
      '  (caller: $suspect, callee: $target) isa call, has started-at $target-call-date;',
      '  $target-call-date > $pattern-call-date;',
      'get $phone-number;'
    ]

    print("\nQuery:\n", "\n".join(query))
    query = "".join(query)

    iterator = transaction.query(query)
    answers = iterator.collect_concepts()
    print('... get answers')
    result = [ answer.value() for answer in answers ]

    print("\nResult:\n", result)


Query:
 match 
  $suspect isa person, has city "London", has age > 50;
  $company isa company, has name "Telecom";
  (customer: $suspect, provider: $company) isa contract;
  $pattern-callee isa person, has age < 20;
  (caller: $suspect, callee: $pattern-callee) isa call, has started-at $pattern-call-date;
  $target isa person, has phone-number $phone-number, has is-customer false;
  (caller: $suspect, callee: $target) isa call, has started-at $target-call-date;
  $target-call-date > $pattern-call-date;
get $phone-number;
... get answers

Result:
 ['+351 272 414 6570', '+30 419 575 7546', '+1 254 875 4647', '+33 614 339 0298', '+86 892 682 0628', '+63 808 497 1769', '+86 922 760 0418', '+86 825 153 5518', '+27 117 258 4149', '+48 894 777 5173', '+351 515 605 7915', '+86 202 257 8619']
